In [6]:
import dash
from dash import dcc  # Updated import for Dash core components
from dash import html  # Updated import for Dash HTML components
from dash.dependencies import Input, Output, State
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
from pykrx import stock
import warnings 
warnings.filterwarnings('ignore')
import yfinance as yf

In [42]:
df = pd.read_excel('df_etf.xlsx', sheet_name ='Sheet1')
df = df.set_index('Date')
df['KODEX 미국10년국채선물']

Date
2012-01-10        NaN
2012-01-11        NaN
2012-01-12        NaN
2012-01-13        NaN
2012-01-16        NaN
               ...   
2022-03-28    11355.0
2022-03-29    11345.0
2022-03-30    11350.0
2022-03-31    11380.0
2022-04-01    11380.0
Name: KODEX 미국10년국채선물, Length: 2517, dtype: float64

### 데이터
- 1번이 좀 오래 걸린당

In [13]:
tickers = stock.get_etf_ticker_list("20240510")

In [17]:
df = stock.get_etf_ohlcv_by_date("20210104", "20240513", "308620")
print(df.head())

                 NAV     시가     고가     저가     종가    거래량       거래대금    기초지수
날짜                                                                        
2021-01-04  11352.01  11370  11370  11295  11335  20354  230731910  220.53
2021-01-05  11401.42  11370  11415  11360  11415  14517  165348740  221.20
2021-01-06  11361.04  11380  11385  11330  11335  18620  211391025  219.78
2021-01-07  11320.68  11365  11365  11315  11340  11049  125233435  219.72
2021-01-08  11317.95  11365  11405  11325  11330  13106  148797780  219.72


In [37]:
tickers = stock.get_etf_ticker_list("20240510")
len(tickers)

857

In [43]:
start_date = "20220402"
end_date = "20240510"
dates = pd.date_range(start=start_date, end=end_date, freq='B')

tickers = stock.get_etf_ticker_list("20240510")
data = []
for ticker in tickers:
    name = stock.get_etf_ticker_name(ticker)
    data.append([ticker, name])

ticker_names = pd.DataFrame(data, columns=['Ticker', 'Name'])
name_dict = ticker_names.set_index('Ticker')['Name'].to_dict()
full_data = pd.DataFrame(index=dates.strftime("%Y%m%d"), columns=tickers)

for date in dates:
    date_str = date.strftime("%Y%m%d")
    daily_data = stock.get_etf_ohlcv_by_ticker(date_str)
    if daily_data is not None and '종가' in daily_data.columns:
        full_data.loc[date_str] = daily_data['종가']
full_data.rename(columns=name_dict, inplace=True)
full_data = full_data.sort_index(axis=1)
full_data.index.name = 'Date'
full_data.index = pd.to_datetime(full_data.index, format='%Y%m%d')
full_data.head()

,1Q 25-08 회사채(A+이상)액티브,1Q K200액티브,1Q 단기금융채액티브,1Q 머니마켓액티브,1Q 차이나H(H),ACE 11월만기자동연장회사채AA-이상액티브,ACE 200,ACE 200TR,ACE 24-12 회사채(AA-이상)액티브,ACE 25-06 회사채(AA-이상)액티브,...,히어로즈 TDF2040액티브,히어로즈 TDF2050액티브,히어로즈 국고채30년액티브,히어로즈 글로벌리츠이지스액티브,히어로즈 단기채권ESG액티브,히어로즈 리츠이지스액티브,히어로즈 머니마켓액티브,히어로즈 미국달러SOFR금리액티브(합성),히어로즈 미국성장기업30액티브,히어로즈 종합채권(AA-이상)액티브
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-04,NaN,NaN,NaN,NaN,NaN,NaN,37265.0,22700.0,NaN,NaN,...,NaN,NaN,NaN,NaN,99925.0,NaN,NaN,NaN,NaN,NaN
2022-04-05,NaN,NaN,NaN,NaN,NaN,NaN,37290.0,22665.0,NaN,NaN,...,NaN,NaN,NaN,NaN,99890.0,NaN,NaN,NaN,NaN,NaN
2022-04-06,NaN,NaN,NaN,NaN,NaN,NaN,36880.0,22415.0,NaN,NaN,...,NaN,NaN,NaN,NaN,99790.0,NaN,NaN,NaN,NaN,NaN
2022-04-07,NaN,NaN,NaN,NaN,NaN,NaN,36375.0,22150.0,NaN,NaN,...,NaN,NaN,NaN,NaN,99800.0,NaN,NaN,NaN,NaN,NaN
2022-04-08,NaN,NaN,NaN,NaN,NaN,NaN,36400.0,22145.0,NaN,NaN,...,NaN,NaN,NaN,NaN,99800.0,NaN,NaN,NaN,NaN,NaN


In [49]:
full_data['KODEX 미국10년국채선물']

Date
2022-04-04    11330.0
2022-04-05    11305.0
2022-04-06    11210.0
2022-04-07    11280.0
2022-04-08    11275.0
               ...   
2024-05-06        NaN
2024-05-07    11150.0
2024-05-08      11210
2024-05-09      11220
2024-05-10      11225
Name: KODEX 미국10년국채선물, Length: 550, dtype: object

In [50]:
matching_columns = [col for col in full_data.columns if col in df.columns]
filtered_data = full_data[matching_columns]
filtered_data

,ARIRANG 200,ARIRANG 200동일가중,ARIRANG 200선물레버리지,ARIRANG 200선물인버스2X,ARIRANG ESG가치주액티브,ARIRANG ESG성장주액티브,ARIRANG KRX300,ARIRANG KS로우볼가중TR,ARIRANG KS로우사이즈가중TR,ARIRANG KS모멘텀가중TR,...,TIGER 코스닥150선물인버스,TIGER 코스피,TIGER 코스피고배당,TIGER 코스피대형주,TIGER 코스피중형주,TIGER 탄소효율그린뉴딜,TIGER 퓨처모빌리티액티브,TIGER 헬스케어,TIGER 현대차그룹+펀더멘털,TIGER 화장품
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-04,37775.0,11055.0,36840.0,4910.0,9005.0,8855.0,17095.0,13275.0,16525.0,15035.0,...,4520.0,28530.0,16260.0,13705.0,16570.0,9040.0,10520.0,31575.0,22520.0,2995.0
2022-04-05,37755.0,11065.0,36820.0,4915.0,8995.0,8880.0,17140.0,13255.0,16570.0,15050.0,...,4490.0,28560.0,16140.0,13720.0,16570.0,9040.0,10565.0,31500.0,22560.0,3015.0
2022-04-06,37335.0,11010.0,36010.0,5020.0,8925.0,8785.0,16975.0,13185.0,16440.0,14925.0,...,4575.0,28360.0,16235.0,13615.0,16585.0,8960.0,10420.0,31100.0,22705.0,2995.0
2022-04-07,36845.0,10850.0,35000.0,5150.0,8805.0,8670.0,16740.0,13025.0,16270.0,14675.0,...,4605.0,27960.0,16240.0,13450.0,16340.0,8840.0,10390.0,30705.0,22425.0,2890.0
2022-04-08,36875.0,10890.0,35040.0,5155.0,8815.0,8700.0,16750.0,13045.0,16265.0,14760.0,...,4565.0,27950.0,16165.0,13430.0,16380.0,8830.0,10565.0,30745.0,22225.0,2865.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,37910.0,10385.0,36790.0,4095.0,8740.0,8865.0,17775.0,14015.0,16760.0,15340.0,...,3475.0,28185.0,14395.0,13600.0,14985.0,9335.0,9525.0,34100.0,27620.0,2830.0
2024-05-08,38020,10440,37030,4065,8760,8900,17805,14080,16795,15415,...,3475,28295,14540,13635,15075,9370,9520,34085,27810,2815


In [51]:
temp = pd.concat([df, filtered_data], axis=0, join='inner')
temp

,ARIRANG 200,ARIRANG 200동일가중,ARIRANG 200선물레버리지,ARIRANG 200선물인버스2X,ARIRANG ESG가치주액티브,ARIRANG ESG성장주액티브,ARIRANG KRX300,ARIRANG KS로우볼가중TR,ARIRANG KS로우사이즈가중TR,ARIRANG KS모멘텀가중TR,...,KODEX 코스피100,KODEX 코스피TR,KODEX 코스피대형주,KODEX 콩선물(H),KODEX 퀄리티Plus,KODEX 탄소효율그린뉴딜,KODEX 필수소비재,KODEX 한국대만IT프리미어,KODEX 헬스케어,KODEX 혁신기술테마액티브
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-10,20817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9030,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-11,20695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9010,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-12,20893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8835,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-13,21042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8720,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-16,20903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8510,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,37910.0,10385.0,36790.0,4095.0,8740.0,8865.0,17775.0,14015.0,16760.0,15340.0,...,28755.0,13555.0,13095.0,13430.0,8705.0,9390.0,6720.0,23255.0,15350.0,13780.0
2024-05-08,38020,10440,37030,4065,8760,8900,17805,14080,16795,15415,...,28850,13595,13145,13270,8765,9390,6680,23405,15370,13840


In [52]:
temp.to_csv('temp.csv', index=True)

In [ ]:
#df2
vix_data = yf.download('^VIX', start='2012-01-10', end='2024-05-10')
vix = vix_data['Adj Close']
vix = pd.DataFrame(vix)
vix = vix.rename(columns={'Adj Close': 'VIX'})
vix

In [41]:
vix.to_csv('vix.csv', index=True)

### 프로젝트 코드
- http://localhost:8050/ 

In [3]:
df = pd.read_excel('df_etf.xlsx', sheet_name ='Sheet1')
df = df.set_index('Date')
temp = pd.read_csv('temp.csv', index_col='Date')
vix = pd.read_csv('vix.csv', index_col='Date')

In [20]:
df.head()

,ARIRANG 200,ARIRANG 200동일가중,ARIRANG 200선물레버리지,ARIRANG 200선물인버스2X,ARIRANG ESG가치주액티브,ARIRANG ESG성장주액티브,ARIRANG ESG우수기업,ARIRANG KRX300,ARIRANG KRX300헬스케어,ARIRANG KS로우볼가중TR,...,KODEX 코스피100,KODEX 코스피TR,KODEX 코스피대형주,KODEX 콩선물(H),KODEX 퀄리티Plus,KODEX 탄소효율그린뉴딜,KODEX 필수소비재,KODEX 한국대만IT프리미어,KODEX 헬스케어,KODEX 혁신기술테마액티브
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-10,20817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9030,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-11,20695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9010,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-12,20893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8835,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-13,21042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8720,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-16,20903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8510,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
temp.head()

,ARIRANG 200,ARIRANG 200동일가중,ARIRANG 200선물레버리지,ARIRANG 200선물인버스2X,ARIRANG ESG가치주액티브,ARIRANG ESG성장주액티브,ARIRANG KRX300,ARIRANG KS로우볼가중TR,ARIRANG KS로우사이즈가중TR,ARIRANG KS모멘텀가중TR,...,KODEX 코스피100,KODEX 코스피TR,KODEX 코스피대형주,KODEX 콩선물(H),KODEX 퀄리티Plus,KODEX 탄소효율그린뉴딜,KODEX 필수소비재,KODEX 한국대만IT프리미어,KODEX 헬스케어,KODEX 혁신기술테마액티브
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-10,20817.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9030.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-11,20695.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9010.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-12,20893.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8835.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-13,21042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8720.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-16,20903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8510.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
vix.head()

,VIX
Date,
2012-01-10,20.690001
2012-01-11,21.049999
2012-01-12,20.469999
2012-01-13,20.910000
2012-01-17,22.200001


### 페어 검증

In [4]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import plotly.graph_objects as go
from statsmodels import regression

def adf_test(temp, critical_value):
    p_value  = adfuller(temp)[1]
    
    if p_value < critical_value:
        print("P-Value of Spread: {}".format(p_value))
        print("TimeSeries Data is Stationary")
        return True
    if p_value > critical_value:
        print("P-Value of Spread: {}".format(p_value))
        print("TimeSeries Data is Non-Stationary")
        return False

df_copy = temp.copy() #df_copy temp 복사한 것 

condition_kodex = [df_copy.columns[i] for i in range(df_copy.shape[1]) if 'KODEX' in df_copy.columns[i]]
df_copy_kodex = df_copy.loc[:, condition_kodex]
df_copy_kodex

,KODEX 200,KODEX 200 중소형,KODEX 200ESG,KODEX 200IT TR,KODEX 200TR,KODEX 200exTOP,KODEX 200가치저변동,KODEX 200동일가중,KODEX 200롱코스닥150숏선물,KODEX 200미국채혼합,...,KODEX 코스피100,KODEX 코스피TR,KODEX 코스피대형주,KODEX 콩선물(H),KODEX 퀄리티Plus,KODEX 탄소효율그린뉴딜,KODEX 필수소비재,KODEX 한국대만IT프리미어,KODEX 헬스케어,KODEX 혁신기술테마액티브
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-10,20834.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9030.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-11,20785.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9010.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-12,20940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8835.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-13,21172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8720.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-16,20951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8510.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,37330.0,13325.0,13635.0,13355.0,12975.0,12060.0,11390.0,11145.0,8625.0,12895.0,...,28755.0,13555.0,13095.0,13430.0,8705.0,9390.0,6720.0,23255.0,15350.0,13780.0
2024-05-08,37460.0,13415.0,13700.0,13345.0,13030.0,12115.0,11500.0,11200.0,8650.0,12940.0,...,28850.0,13595.0,13145.0,13270.0,8765.0,9390.0,6680.0,23405.0,15370.0,13840.0


In [77]:
etf_1 = 'KODEX 코스피'
etf_2 = 'KODEX 코스닥150'

start_date = '2024-01-01'
end_date = '2024-05-01'

df_copy_kodex_sample = df_copy_kodex.loc[start_date:end_date,[etf_1, etf_2]].dropna()

# Regression

regression_model = regression.linear_model.OLS(df_copy_kodex_sample[etf_1], df_copy_kodex_sample[etf_2])

fitted_model = regression_model.fit()

print("Ratio of {}:  {}".format(etf_2, fitted_model.params[0]))

spread = df_copy_kodex_sample[etf_1] - fitted_model.params[0] * df_copy_kodex_sample[etf_2]
adf_test(spread, critical_value = 0.05)

data_mean = [spread.mean() for i in range(spread.shape[0])]

# Make Graph

trace1 = go.Scatter(x = spread.index, 
                   y = spread,
                   mode ='lines',
                   name = 'Spread')
trace2 = go.Scatter(x = spread.index,
                   y = data_mean,
                   mode = 'lines',
                   name = 'Spread Mean')
layout = go.Layout(title = '{} & {}'.format(etf_1, etf_2),
                  yaxis = {'tickformat':','})
fig = go.Figure(data= [trace1, trace2], layout = layout)
fig.update_layout(template = 'seaborn')
fig.show()

Ratio of KODEX 코스닥150:  1.945856428602182
P-Value of Spread: 0.1253852714546111
TimeSeries Data is Non-Stationary


In [81]:
etf_1 = 'KODEX 코스피'
etf_2 = 'KODEX 반도체'

start_date = '2023-07-01'
end_date = '2024-03-01'

df_copy_kodex_sample = df_copy_kodex.loc[start_date:end_date,[etf_1, etf_2]].dropna()

# Regression

regression_model = regression.linear_model.OLS(df_copy_kodex_sample[etf_1], df_copy_kodex_sample[etf_2])

fitted_model = regression_model.fit()

print("Ratio of {}:  {}".format(etf_2, fitted_model.params[0]))

spread = df_copy_kodex_sample[etf_1] - fitted_model.params[0] * df_copy_kodex_sample[etf_2]
adf_test(spread, critical_value = 0.05)

data_mean = [spread.mean() for i in range(spread.shape[0])]

# Make Graph

trace1 = go.Scatter(x = spread.index, 
                   y = spread,
                   mode ='lines',
                   name = 'Spread')
trace2 = go.Scatter(x = spread.index,
                   y = data_mean,
                   mode = 'lines',
                   name = 'Spread Mean')
layout = go.Layout(title = '{} & {}'.format(etf_1, etf_2),
                  yaxis = {'tickformat':','})
fig = go.Figure(data= [trace1, trace2], layout = layout)
fig.update_layout(template = 'seaborn')
fig.show()

Ratio of KODEX 반도체:  0.7643175426150566
P-Value of Spread: 0.041457606496010824
TimeSeries Data is Stationary


In [101]:
def adf_test(df, critical_value):
    p_value  = adfuller(df)[1]
    
    if p_value < critical_value:
        print("P-Value of Spread: {}".format(p_value))
        print("TimeSeries Data is Stationary")
        return True
    if p_value > critical_value:
        print("P-Value of Spread: {}".format(p_value))
        print("TimeSeries Data is Non-Stationary")
        return False

condition_tiger = [df_copy.columns[i] for i in range(df_copy.shape[1]) if 'TIGER' in df_copy.columns[i]]
df_copy_tiger = df_copy.loc[:, condition_tiger]

etf_1 = 'TIGER 차이나전기차SOLACTIVE'
etf_2 = 'TIGER 2차전지테마'

start_date = '2023-05-01'
end_date = '2024-05-01'

df_copy_tiger_sample = df_copy_tiger.loc[start_date:end_date,[etf_1, etf_2]].dropna()

# Regression

regression_model = regression.linear_model.OLS(df_copy_tiger_sample[etf_1], df_copy_tiger_sample[etf_2])

fitted_model = regression_model.fit()

print("Ratio of {}:  {}".format(etf_2, fitted_model.params[0]))

spread = df_copy_tiger_sample[etf_1] - fitted_model.params[0] * df_copy_tiger_sample[etf_2]
adf_test(spread, critical_value = 0.05)

data_mean = [spread.mean() for i in range(spread.shape[0])]

# Make Graph

trace1 = go.Scatter(x = spread.index, 
                   y = spread,
                   mode ='lines',
                   name = 'Spread')
trace2 = go.Scatter(x = spread.index,
                   y = data_mean,
                   mode = 'lines',
                   name = 'Spread Mean')
layout = go.Layout(title = '{} & {}'.format(etf_1, etf_2),
                  yaxis = {'tickformat':','})
fig = go.Figure(data= [trace1, trace2], layout = layout)
fig.update_layout(template = 'seaborn')
fig.show()

Ratio of TIGER 2차전지테마:  0.30601226836987644
P-Value of Spread: 0.02449954156884496
TimeSeries Data is Stationary


In [25]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

df_copy_tiger_sample1 = df_copy_tiger.loc[:, [etf_1, etf_2]].dropna()
df_copy_tiger_sample1.iplot(kind='scatter')

In [26]:
etf_1 = 'KODEX 국고채3년'
etf_2 = 'KODEX 국채선물10년'

start_date = '2023-01-01'
end_date = '2024-03-31'

df_copy_kodex_sample = df_copy_kodex.loc[start_date:end_date,[etf_1, etf_2]].dropna()

# Regression

regression_model = regression.linear_model.OLS(df_copy_kodex_sample[etf_1], df_copy_kodex_sample[etf_2])

fitted_model = regression_model.fit()

print("Ratio of {}:  {}".format(etf_2, fitted_model.params[0]))

#spread = df_copy_kodex_sample[etf_1] - fitted_model.params[0] * df_copy_kodex_sample[etf_2]
spread = df_copy_kodex_sample[etf_1] - fitted_model.params[0] * df_copy_kodex_sample[etf_2]

adf_test(spread, critical_value = 0.05)

data_mean = [spread.mean() for i in range(spread.shape[0])]

# Make Graph

trace1 = go.Scatter(x = spread.index, 
                   y = spread,
                   mode ='lines',
                   name = 'Spread')
trace2 = go.Scatter(x = spread.index,
                   y = data_mean,
                   mode = 'lines',
                   name = 'Spread Mean')
layout = go.Layout(title = 'Spread of {} & {}'.format(etf_1, etf_2),
                  yaxis = {'tickformat':','})
fig = go.Figure(data= [trace1, trace2], layout = layout)
fig.update_layout(template = 'seaborn')
fig.show()

Ratio of KODEX 국채선물10년:  0.9058048015744873
P-Value of Spread: 0.41361900782163963
TimeSeries Data is Non-Stationary


In [27]:
etf_1 = 'KODEX 은행'
etf_2 = 'KODEX 국채선물10년'

start_date = '2021-01-01'
end_date = '2022-03-31'

df_copy_kodex_sample = df_copy_kodex.loc[start_date:end_date,[etf_1, etf_2]].dropna()

# Regression

regression_model = regression.linear_model.OLS(df_copy_kodex_sample[etf_1], df_copy_kodex_sample[etf_2])

fitted_model = regression_model.fit()

print("Ratio of {}:  {}".format(etf_2, fitted_model.params[0]))

#spread = df_copy_kodex_sample[etf_1] - fitted_model.params[0] * df_copy_kodex_sample[etf_2]
spread = df_copy_kodex_sample[etf_1] - fitted_model.params[0] * df_copy_kodex_sample[etf_2]

adf_test(spread, critical_value = 0.05)

data_mean = [spread.mean() for i in range(spread.shape[0])]

# Make Graph

trace1 = go.Scatter(x = spread.index, 
                   y = spread,
                   mode ='lines',
                   name = 'Spread')
trace2 = go.Scatter(x = spread.index,
                   y = data_mean,
                   mode = 'lines',
                   name = 'Spread Mean')
layout = go.Layout(title = 'Spread of {} & {}'.format(etf_1, etf_2),
                  yaxis = {'tickformat':','})
fig = go.Figure(data= [trace1, trace2], layout = layout)
fig.update_layout(template = 'seaborn')
fig.show()

Ratio of KODEX 국채선물10년:  0.10574214330683505
P-Value of Spread: 0.4106016202305863
TimeSeries Data is Non-Stationary


### 대시보드

In [3]:
# Datetime 전환 필요
df = pd.read_excel('df_etf.xlsx', sheet_name='Sheet1')
df['Date'] = pd.to_datetime(df['Date'])  
df = df.set_index('Date') 
temp = pd.read_csv('temp.csv')
temp['Date'] = pd.to_datetime(temp['Date'])
temp = temp.set_index('Date')
vix = pd.read_csv('vix.csv')
vix['Date'] = pd.to_datetime(vix['Date'])
vix = vix.set_index('Date')

In [57]:
temp

,ARIRANG 200,ARIRANG 200동일가중,ARIRANG 200선물레버리지,ARIRANG 200선물인버스2X,ARIRANG ESG가치주액티브,ARIRANG ESG성장주액티브,ARIRANG KRX300,ARIRANG KS로우볼가중TR,ARIRANG KS로우사이즈가중TR,ARIRANG KS모멘텀가중TR,...,KODEX 코스피100,KODEX 코스피TR,KODEX 코스피대형주,KODEX 콩선물(H),KODEX 퀄리티Plus,KODEX 탄소효율그린뉴딜,KODEX 필수소비재,KODEX 한국대만IT프리미어,KODEX 헬스케어,KODEX 혁신기술테마액티브
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-10,20817.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9030.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-11,20695.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9010.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-12,20893.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8835.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-13,21042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8720.0,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-16,20903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8510.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,37910.0,10385.0,36790.0,4095.0,8740.0,8865.0,17775.0,14015.0,16760.0,15340.0,...,28755.0,13555.0,13095.0,13430.0,8705.0,9390.0,6720.0,23255.0,15350.0,13780.0
2024-05-08,38020.0,10440.0,37030.0,4065.0,8760.0,8900.0,17805.0,14080.0,16795.0,15415.0,...,28850.0,13595.0,13145.0,13270.0,8765.0,9390.0,6680.0,23405.0,15370.0,13840.0


In [55]:
vix

,VIX
Date,
2012-01-10,20.690001
2012-01-11,21.049999
2012-01-12,20.469999
2012-01-13,20.910000
2012-01-17,22.200001
...,...
2024-05-03,13.490000
2024-05-06,13.490000
2024-05-07,13.230000


In [4]:
def RSI(df, rsi_period):
    
    df_close_copy = df.copy()
    
    df_close_copy['Diff'] = df_close_copy.diff()
    
    df_close_copy['UP'] = np.where(df_close_copy['Diff']>0, df_close_copy['Diff'], 0)
    
    df_close_copy['DOWN'] = np.where(df_close_copy['Diff']<0, abs(df_close_copy['Diff']),0)
    
    df_close_copy['AU'] = df_close_copy['UP'].rolling(rsi_period).mean()
    
    df_close_copy['AD'] = df_close_copy['DOWN'].rolling(rsi_period).mean()
    
    df_close_copy['RS'] = df_close_copy['AU'] / df_close_copy['AD']
    
    df_close_copy['RSI'] = round(df_close_copy['RS'] / (1 + df_close_copy['RS']) * 100,2)
    
    return df_close_copy[['RSI']]

def macdOscillator_fn(df, short_N, long_N, signal_N):
    
    df_close_copy = df.copy()
    
    df_close_copy['Short'] = df_close_copy[df_close_copy.columns[0]].ewm(span = short_N, adjust = False).mean()
    
    df_close_copy['Long'] = df_close_copy[df_close_copy.columns[0]].ewm(span = long_N, adjust = False).mean()
    
    df_close_copy['MACD'] = df_close_copy['Short'] - df_close_copy['Long']
    
    df_close_copy['Signal'] = df_close_copy['MACD'].ewm(span = signal_N, adjust = False).mean()
    
    df_close_copy['MACD Oscillator'] = df_close_copy['MACD'] - df_close_copy['Signal']
    
    return df_close_copy[['MACD', 'Signal', 'MACD Oscillator']]


df_1 = temp #수정

df_2 = vix #수정

df_col_list = pd.read_excel('df_etf.xlsx', sheet_name ='Sheet3')

df = pd.concat([df_1, df_2], axis = 1, join = 'outer')

df_copy = df.copy()

stock_index_col = df_col_list['Stock Index'].dropna().values

macro_index_col = df_col_list['Macro'].dropna().values

sector_index_col = df_col_list['Sector'].dropna().values

options_stock_index = [{'label':i, 'value':i} for i in stock_index_col]

options_macro_index = [{'label':i, 'value':i} for i in macro_index_col]

options_sector_index = [{'label':i, 'value':i} for i in sector_index_col]


#%% Layout

app = dash.Dash()

app.layout = html.Div([
    
    ##### Layout1
    html.Div(id = 'div1',
             children = [
                 
                 html.H3('Stock Index DashBoard', style = {'textAlign':'center'}),
                 html.Div(id = 'div1 input',
                          children = [
                              html.H5('Select ETF'),
                              dcc.Dropdown(id = 'div1 etf col',
                                           options = options_stock_index,
                                           multi = False,
                                           value = 'TIGER 200',
                                           style = {'width':'60%', 'fontSize':15}),
                              dcc.Input(id = 'div1 start date',
                                        value = '20190101',
                                        style = {'fontSize':15}),
                              dcc.Input(id = 'div1 end date',
                                        value = '20220101',
                                        style = {'fontSize':15}),
                              html.Br(),
                              html.Br(),
                              dcc.RadioItems(id = 'div1 radio items',
                                             options = [{'label':'RSI','value':'RSI'}, {'label':'MACD','value':'MACD'}],
                                             value = 'MACD'),
                              
                              html.Br(),
                              html.Button(id = 'div1 button',
                                          n_clicks = 0,
                                          children = 'Submit',
                                          style = {'fontSize':15})
                                  ]
                          ),
                 html.Div(id = 'div1 Graph',
                          children = [
                              dcc.Graph(id = 'div1 graph1'),
                              dcc.Graph(id = 'div1 graph2')
                              ])
                 
                     ], style = {'display':'inline-block', 'width':'50%', 'verticalAlign':'top'}),
    
    
    ##### Layout2
    html.Div(id = 'div2',
             children = [
                 
                 html.H3('Macro Index DashBoard', style = {'textAlign':'center'}),
                 html.Div(id = 'div2 input',
                          children = [
                              html.H5('Select ETF'),
                              dcc.Dropdown(id = 'div2 etf col',
                                           options = options_macro_index,
                                           multi = False,
                                           value = 'TIGER 구리실물',
                                           style = {'width':'60%', 'fontSize':15}),
                              dcc.Input(id = 'div2 start date',
                                        value = '20190101',
                                        style = {'fontSize':15}),
                              dcc.Input(id = 'div2 end date',
                                        value = '20220101',
                                        style = {'fontSize':15}),
                              html.Br(),
                              html.Br(),
                              dcc.RadioItems(id = 'div2 radio items',
                                             options = [{'label':'RSI','value':'RSI'}, {'label':'MACD','value':'MACD'}],
                                             value = 'MACD'),
                              
                              html.Br(),
                              html.Button(id = 'div2 button',
                                          n_clicks = 0,
                                          children = 'Submit',
                                          style = {'fontSize':15})
                                  ]
                          ),
                 html.Div(id = 'div2 Graph',
                          children = [
                              dcc.Graph(id = 'div2 graph1'),
                              dcc.Graph(id = 'div2 graph2')
                              ])
                 
                 
                 
                 ], style = {'display':'inline-block', 'width':'50%', 'verticalAlign':'top'}),
    
    
    ##### Layout3
html.Div(id='div3',
         children=[
             html.H3('Return HeatMap', style={'textAlign': 'center'}),
             html.Div(id='div3 input',
                      children=[
                          html.H5('Select Index'),
                          dcc.RadioItems(id='div3 radio items',
                                         options=[{'label': 'Stock Index', 'value': 'Stock Index'},
                                                  {'label': 'Macro', 'value': 'Macro'},
                                                  {'label': 'Sector', 'value': 'Sector'}],
                                         value='Sector'),

                          html.Br(),

                          dcc.Input(id='div3 date',
                                    value='20240510',
                                    style={'fontSize': 15}),
                          html.Br(),
                          html.Br(),

                          html.Button(id='div3 button',
                                      n_clicks=0,
                                      children='Submit',
                                      style={'fontSize': 15})
                      ]),

             html.Br(),
             html.Br(),

             html.Div(id='div3 Graph',
                      children=[
                          html.Div(id='div3 date layout',
                                   children=['None']),
                          html.Br(),
                          dcc.Graph(id='div3 graph',
                                    )

                      ])

         ], style={'display': 'block', 'width': '100%', 'verticalAlign': 'top'}),

##### Layout4
html.Div(id='div4',
         children=[
             html.H3('Macro & Sector', style={'textAlign': 'center'}),
             html.Div(id='div4 input',
                      children=[
                          html.H5('Select Macro & Sector'),
                          dcc.Dropdown(id='div4 macro etf col',
                                       options=options_macro_index,
                                       multi=True,
                                       value=['KODEX 콩선물(H)', 'KODEX 국채선물10년인버스'],
                                       style={'width': '60%', 'fontSize': 15}),
                          dcc.Dropdown(id='div4 sector etf col',
                                       options=options_sector_index,
                                       multi=True,
                                       value=['TIGER 200 경기소비재', 'TIGER 200 금융'],
                                       style={'width': '60%', 'fontSize': 15}),
                          dcc.Input(id='div4 start date',
                                    value='20210601',
                                    style={'fontSize': 15}),
                          dcc.Input(id='div4 end date',
                                    value='20220101',
                                    style={'fontSize': 15}),
                          html.Br(),
                          html.Br(),

                          html.Button(id='div4 button',
                                      n_clicks=0,
                                      children='Submit',
                                      style={'fontSize': 15})
                      ]),
             dcc.Graph(id='div4 graph')

         ], style={'display': 'block', 'width': '100%', 'verticalAlign': 'top'})])

##### Callback Funtion
# Div1 Callback Function
@app.callback([Output('div1 graph1', 'figure'),
               Output('div1 graph2', 'figure')],
              [Input('div1 button', 'n_clicks')],
              [State('div1 etf col', 'value'),
               State('div1 start date', 'value'),
               State('div1 end date', 'value'),
               State('div1 radio items', 'value')])
def update_div1_graph(n_clicks, etf_col, start_date, end_date, radio_item):
    start_year = int(start_date[:4])
    start_month = int(start_date[4:6])
    start_day = int(start_date[6:8])
    start = datetime(start_year, start_month, start_day)
    
    end_year = int(end_date[:4])
    end_month = int(end_date[4:6])
    end_day = int(end_date[6:8])
    end = datetime(end_year, end_month, end_day)
    
    df_trace = df_copy.loc[start:end, [etf_col]].dropna()
    
    start_date_format = '{}Y - {}M - {}D'.format(df_trace.index[0].year, df_trace.index[0].month, df_trace.index[0].day)
    end_date_format = '{}Y - {}M - {}D'.format(df_trace.index[-1].year, df_trace.index[-1].month, df_trace.index[-1].day)
    
    trace1 = go.Scatter(x=df_trace.index,
                        y = df_trace[etf_col],
                        mode = 'lines')
    layout1 = go.Layout(title = '{}'.format(etf_col),
                        xaxis = {'title':'{} ~ {}'.format(start_date_format, end_date_format)},
                        yaxis = {'tickformat':','})
    
    fig1 = go.Figure(data = trace1, layout = layout1)
    fig1.update_layout(template = 'seaborn')
    
    if radio_item =='MACD':
        short_N = 9
        long_N = 26
        signal_N = 13
        df_macd = macdOscillator_fn(df = df_trace, short_N=short_N, long_N = long_N, signal_N = signal_N)
        
        trace2_1 = go.Scatter(x = df_macd.index,
                              y = df_macd['MACD'],
                              mode = 'lines',
                              name = 'MACD')
        trace2_2 = go.Scatter(x = df_macd.index,
                              y = df_macd['Signal'],
                              mode = 'lines',
                              name = 'Signal')
        trace2_3 = go.Scatter(x = df_macd.index,
                              y = df_macd['MACD Oscillator'],
                              mode = 'lines',
                              name = 'MACD Oscillator')
        layout2 = go.Layout(title = 'MACD')
        fig2 = go.Figure(data=[trace2_1, trace2_2, trace2_3], layout = layout2)
        fig2.update_layout(template = 'seaborn')
    else:
        df_rsi = RSI(df = df_trace, rsi_period = 22)
        trace2_1 = go.Scatter(x = df_rsi.index,
                              y = df_rsi['RSI'],
                              mode = 'lines',
                              name = 'RSI')
        trace2_2 = go.Scatter(x = df_rsi.index,
                              y = (df_rsi['RSI']/df_rsi['RSI']).fillna(1)*30,
                              line = dict(color = 'red', dash = 'dash'),
                              name = 'Lower Level')
        trace2_3 = go.Scatter(x = df_rsi.index,
                              y = (df_rsi['RSI']/df_rsi['RSI']).fillna(1)*70,
                              line = dict(color = 'red', dash = 'dash'),
                              name = 'Upper Level')
        layout2 = go.Layout(title = 'RSI')
        fig2 = go.Figure(data = [trace2_1, trace2_2, trace2_3], layout = layout2)
        fig2.update_layout(template = 'seaborn')
    
    return fig1, fig2

# Div2 Callback Function
@app.callback([Output('div2 graph1', 'figure'),
               Output('div2 graph2', 'figure')],
              [Input('div2 button', 'n_clicks')],
              [State('div2 etf col', 'value'),
               State('div2 start date', 'value'),
               State('div2 end date', 'value'),
               State('div2 radio items', 'value')])
def update_div2_graph(n_clicks, etf_col, start_date, end_date, radio_item):
    start_year = int(start_date[:4])
    start_month = int(start_date[4:6])
    start_day = int(start_date[6:8])
    start = datetime(start_year, start_month, start_day)
    
    end_year = int(end_date[:4])
    end_month = int(end_date[4:6])
    end_day = int(end_date[6:8])
    end = datetime(end_year, end_month, end_day)
    
    df_trace = df_copy.loc[start:end, [etf_col]].dropna()
    
    start_date_format = '{}Y - {}M - {}D'.format(df_trace.index[0].year, df_trace.index[0].month, df_trace.index[0].day)
    end_date_format = '{}Y - {}M - {}D'.format(df_trace.index[-1].year, df_trace.index[-1].month, df_trace.index[-1].day)
    
    trace1 = go.Scatter(x=df_trace.index,
                        y = df_trace[etf_col],
                        mode = 'lines')
    layout1 = go.Layout(title = '{}'.format(etf_col),
                        xaxis = {'title':'{} ~ {}'.format(start_date_format, end_date_format)},
                        yaxis = {'tickformat':','})
    
    fig1 = go.Figure(data = trace1, layout = layout1)
    fig1.update_layout(template = 'seaborn')
    
    if radio_item =='MACD':
        short_N = 9
        long_N = 26
        signal_N = 13
        df_macd = macdOscillator_fn(df = df_trace, short_N=short_N, long_N = long_N, signal_N = signal_N)
        
        trace2_1 = go.Scatter(x = df_macd.index,
                              y = df_macd['MACD'],
                              mode = 'lines',
                              name = 'MACD')
        trace2_2 = go.Scatter(x = df_macd.index,
                              y = df_macd['Signal'],
                              mode = 'lines',
                              name = 'Signal')
        trace2_3 = go.Scatter(x = df_macd.index,
                              y = df_macd['MACD Oscillator'],
                              mode = 'lines',
                              name = 'MACD Oscillator')
        layout2 = go.Layout(title = 'MACD')
        fig2 = go.Figure(data=[trace2_1, trace2_2, trace2_3], layout = layout2)
        fig2.update_layout(template = 'seaborn')
    else:
        df_rsi = RSI(df = df_trace, rsi_period = 22)
        trace2_1 = go.Scatter(x = df_rsi.index,
                              y = df_rsi['RSI'],
                              mode = 'lines',
                              name = 'RSI')
        trace2_2 = go.Scatter(x = df_rsi.index,
                              y = (df_rsi['RSI']/df_rsi['RSI']).fillna(1)*30,
                              line = dict(color = 'red', dash = 'dash'),
                              name = 'Lower Level')
        trace2_3 = go.Scatter(x = df_rsi.index,
                              y = (df_rsi['RSI']/df_rsi['RSI']).fillna(1)*70,
                              line = dict(color = 'red', dash = 'dash'),
                              name = 'Upper Level')
        layout2 = go.Layout(title = 'RSI')
        fig2 = go.Figure(data = [trace2_1, trace2_2, trace2_3], layout = layout2)
        fig2.update_layout(template = 'seaborn')
    
    return fig1, fig2

# Div3 Callback Function
@app.callback([Output('div3 graph', 'figure'),
               Output('div3 date layout', 'children')],
              [Input('div3 button', 'n_clicks')],
              [State('div3 radio items', 'value'),
               State('div3 date', 'value')])
def update_div3_graph(n_clicks, radio_item, date):
    
    date_year = int(date[:4])
    date_month = int(date[4:6])
    date_day = int(date[6:8])
    date = datetime(date_year, date_month, date_day)
    
    if radio_item =='Stock Index':
        col_set = stock_index_col
    elif radio_item == 'Sector':
        col_set = sector_index_col
    elif radio_item == 'Macro':
        col_set = macro_index_col[:-1]
    
    df_trace = df_copy.loc[:date, col_set] ##
    
    date_format = '{}Y - {}M - {}D'.format(df_trace.index[-1].year, df_trace.index[-1].month, df_trace.index[-1].day)
    
    result_date_format = 'Reference Date: {}'.format(date_format)
    
    df_trace = df_trace.sort_index(ascending = False)
    
    daily_return  = round(df_trace.pct_change(periods = -1).iloc[0]*100, 1)
    weekly_return = round(df_trace.pct_change(periods = -5).iloc[0]*100, 1)
    monthly_return = round(df_trace.pct_change(periods = -22).iloc[0]*100, 1)
    
    ##### Make DataFrame of Return
    df_return = pd.DataFrame()
    
    df_return['Daily Return'] = daily_return
    df_return['Weekly Return'] = weekly_return
    df_return['Monthly Return'] = monthly_return
    
    #### Make HeatMap
    
    x = ['{}'.format(i) for i in df_return.columns]
    
    y = ['{}'.format(i) for i in df_return.index]
    
    z = df_return.values
    
    trace = ff.create_annotated_heatmap(z, x = x, y = y, colorscale = 'ylgnbu')
    
    layout = go.Layout(title = 'Heat Map of {}'.format(radio_item))
    
    fig = go.Figure(data = trace, layout = layout)
    
    fig.update_layout(
        autosize = False,
        width = 600,
        height = 900
        )
    
    return fig, result_date_format

# Div4 Callback Function
@app.callback(Output('div4 graph', 'figure'),
              [Input('div4 button', 'n_clicks')],
              [State('div4 macro etf col', 'value'),
               State('div4 sector etf col', 'value'),
               State('div4 start date', 'value'),
               State('div4 end date', 'value')])
def update_div4_graph(n_clicks, macro_col, sector_col, start_date, end_date):
    
    start_year = int(start_date[:4])
    start_month = int(start_date[4:6])
    start_day = int(start_date[6:8])
    start = datetime(start_year, start_month, start_day)
    
    end_year = int(end_date[:4])
    end_month = int(end_date[4:6])
    end_day = int(end_date[6:8])
    end = datetime(end_year, end_month, end_day)
    
    ### Set DataFrame
    
    col_list = macro_col + sector_col
    
    df_trace = df_copy.loc[start:end, col_list].dropna()
    
    start_date_format = '{}Y - {}M - {}D'.format(df_trace.index[0].year, df_trace.index[0].month, df_trace.index[0].day)
    end_date_format = '{}Y - {}M - {}D'.format(df_trace.index[-1].year, df_trace.index[-1].month, df_trace.index[-1].day)
    
    data = []
    
    for col in df_trace.columns:
        trace = go.Scatter(x = df_trace.index,
                           y = df_trace[col]/df_trace[col][0],
                           mode = 'lines',
                           name = '{}'.format(col))
        data.append(trace)
    
    layout = go.Layout(title = 'Macro & Sector Standard Price Graph',
                       xaxis = {'title':'{} ~ {}'.format(start_date_format, end_date_format)},
                       yaxis = {'tickformat':','})
    fig = go.Figure(data = data, layout = layout)
    fig.update_layout(template = 'seaborn')
    
    return fig

In [5]:
import socket #포트 충돌 방지 추가 코드
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

free_port = find_free_port()
app.run(port=free_port)


if __name__=='__main__':
    app.run()

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [64]:
#http://localhost:8050/